# Auditoría de integridad (ejemplo)

Este notebook sirve como ejemplo para **leer auditorías** (salidas de `hdfs fsck`) y construir una **tabla resumen**.

✅ Tarea del alumnado: adaptar las rutas y completar los scripts para que las auditorías se guarden en un directorio accesible desde Jupyter.


## 1) Configuración de rutas

En el entorno del aula, el NameNode monta un volumen (por ejemplo `./notebooks` → `/media/notebooks`).
La idea es que el script `30_fsck_audit.sh` deje copias de auditoría en una ruta que Jupyter pueda leer.


In [ ]:
from pathlib import Path
import pandas as pd
import re

# TODO: ajusta esta ruta según tu despliegue
AUDIT_DIR = Path('/media/notebooks/audit/fsck')  # ejemplo
print('AUDIT_DIR exists:', AUDIT_DIR.exists())
sorted(AUDIT_DIR.glob('*'))[:5]


## 2) Parseo básico de fsck

Contabilizamos palabras clave típicas:
- `CORRUPT`
- `MISSING`
- `Under replicated`


In [ ]:
def parse_fsck_text(text: str):
    return {
        'CORRUPT': len(re.findall(r'\\bCORRUPT\\b', text, flags=re.IGNORECASE)),
        'MISSING': len(re.findall(r'\\bMISSING\\b', text, flags=re.IGNORECASE)),
        'UNDER_REPLICATED': len(re.findall(r'Under replicated', text, flags=re.IGNORECASE)),
    }

rows = []
for dt_dir in sorted(AUDIT_DIR.glob('*')):
    if not dt_dir.is_dir():
        continue
    fsck_file = dt_dir / 'fsck_data.txt'
    if not fsck_file.exists():
        continue
    text = fsck_file.read_text(encoding='utf-8', errors='ignore')
    metrics = parse_fsck_text(text)
    metrics['dt'] = dt_dir.name
    rows.append(metrics)

df = pd.DataFrame(rows).sort_values('dt') if rows else pd.DataFrame(columns=['dt','CORRUPT','MISSING','UNDER_REPLICATED'])
df


## 3) Exportación

Exporta la tabla a CSV para incluirla en el informe.


In [ ]:
out = Path('./audit_summary.csv')
df.to_csv(out, index=False)
out
